# Stock Price Prediction

## Time Series Plots

In [21]:
import quandl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
oildata = quandl.get('NSE/OIL')

In [23]:
oildata.head()

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
Date,,,,,,,
2009-09-30,1096.0,1156.7,1090.0,1135.00,1141.20,19748012.0,223877.07
2009-10-01,1102.0,1173.7,1102.0,1167.00,1166.35,3074254.0,35463.78
2009-10-05,1152.0,1165.9,1136.6,1143.00,1140.55,919832.0,10581.13
2009-10-06,1149.8,1157.2,1132.1,1143.30,1144.90,627957.0,7185.90
2009-10-07,1153.8,1160.7,1140.0,1141.45,1141.60,698216.0,8032.98


In [31]:
oildata.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2271 entries, 2009-09-30 to 2018-11-26
Data columns (total 7 columns):
Open                    2271 non-null float64
High                    2271 non-null float64
Low                     2271 non-null float64
Last                    2271 non-null float64
Close                   2271 non-null float64
Total Trade Quantity    2271 non-null float64
Turnover (Lacs)         2271 non-null float64
dtypes: float64(7)
memory usage: 221.9 KB


In [48]:
r_oildata = oildata.reset_index()
oildata['Total Trade Quantity'].resample('A')
#plot()
#plt.ylabel('Oil Stock Log (Volume)')
#plt.show()


DatetimeIndexResampler [freq=<YearEnd: month=12>, axis=0, closed=right, label=right, convention=start, base=0]

In [30]:
oildata.describe()

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
count,2271.000000,2271.000000,2271.000000,2271.000000,2271.000000,2.271000e+03,2271.000000
mean,657.333443,666.695707,648.174857,656.739960,656.807904,3.966499e+05,1873.164055
std,400.016431,406.511080,394.628845,400.134994,400.166816,6.674977e+05,5208.779515
min,186.600000,195.400000,181.000000,187.400000,186.900000,1.921000e+03,21.520000
25%,369.075000,373.650000,364.000000,368.950000,368.800000,9.717750e+04,702.585000
50%,479.300000,484.500000,472.500000,478.050000,477.950000,2.579160e+05,1251.660000
75%,1137.525000,1154.750000,1125.000000,1141.725000,1141.325000,5.041550e+05,2133.045000
max,1604.000000,1630.000000,1585.000000,1624.000000,1619.950000,1.974801e+07,223877.070000


In [ ]:
oildata['Total Trafe']